In [1]:
import pandas as pd
import numpy as np
from os.path import exists
from tqdm import tqdm
import re
import datatable as dt
# Variables that contains the file location
from files import *
from functions import *
from sklearn.preprocessing import normalize

In [2]:
# if we modify the file we need to reload it with this
import importlib
import functions #import the module here, so that it can be reloaded.
importlib.reload(functions)

<module 'functions' from '/Users/eduardohdz/Documents/JKU/Semester1/MMSR_REPO/project/functions.py'>

In [3]:
genres  = dt.fread(file_genres_2).to_pandas().set_index('id')
info  = dt.fread(file_info_2).to_pandas().set_index('id')

In [4]:
info.head()

,artist,song,album_name
id,,,
0009fFIM1eYThaPg,Cheryl,Rain on Me,3 Words
0010xmHR6UICBOYT,Oddisee,After Thoughts,The Beauty in All
002Jyd0vN4HyCpqL,Blue Öyster Cult,ME 262,Secret Treaties
006TYKNjNxWjfKjy,Rhapsody,Flames of Revenge,Legendary Years (Re-Recorded)
007LIJOPQ4Sb98qV,The Chameleons,Nostalgia,What Does Anything Mean? Basically (2009 Remas...


In [5]:
tf_idf  = dt.fread(file_tfidf_2).to_pandas().set_index('id')
word2vec  = dt.fread(file_word2vec_2, header=True).to_pandas().set_index('id')
bert = dt.fread(file_bert_2,header=True).to_pandas().set_index('id')

In [6]:
blf_correlation  = dt.fread(file_blf_correlation).to_pandas().set_index('id')
blf_deltaspectral  = dt.fread(file_blf_deltaspectral).to_pandas().set_index('id')
blf_spectral  = dt.fread(file_blf_spectral).to_pandas().set_index('id')
blf_spectralcontrast  = dt.fread(file_blf_spectralcontrast).to_pandas().set_index('id')
blf_vardeltaspectral  = dt.fread(file_blf_vardeltaspectral).to_pandas().set_index('id')
essentia  = dt.fread(file_essentia).to_pandas().set_index('id')
mfcc_bow  = dt.fread(file_mfcc_bow).to_pandas().set_index('id')
mfcc_stats  = dt.fread(file_mfcc_stats).to_pandas().set_index('id')

In [7]:
blf_logfluc  = dt.fread(file_blf_logfluc)
# This is done because in the csv it has an extra column name, 
# so in case someone with the original dataset tries to run it, it fixes that error
# It looks weird, but it is because first i am loading the data into datatable and then pass it to dataframe
new_cols = ['id']
new_cols.extend(list(blf_logfluc.names[2:]))
new_cols = tuple(new_cols)
del blf_logfluc[:, -1]

blf_logfluc.names = new_cols
blf_logfluc = blf_logfluc.to_pandas()
blf_logfluc.set_index('id', inplace=True)

In [8]:
incp  = dt.fread(file_incp).to_pandas().set_index('id')
resnet  = dt.fread(file_resnet).to_pandas().set_index('id')
vgg19  = dt.fread(file_vgg19).to_pandas().set_index('id')

In [9]:
datasets = {
    "tfidf" : tf_idf,
    "word2vec" : word2vec,
    "bert" : bert,
    "mfcc_bow" : mfcc_bow,
    "mfcc_stats" : mfcc_stats,
    "essentia" : essentia,
    "blf_delta_spectral" : blf_deltaspectral,
    "blf_correlation" : blf_correlation,
    "blf_logfluc" : blf_logfluc,
    "blf_spectral" : blf_spectral,
    "blf_spectral_contrast" : blf_spectralcontrast,
    "blf_vardelta_spectral" : blf_vardeltaspectral,
    "incp" : incp,
    "vgg19" : vgg19,
    "resnet" : resnet
}

In [10]:
# Audio Features
print("Lyrics Features\n")
## Mel Frequency Cepstral Coefficients (MFFCs)
print("tfidf dataset:   ", tf_idf.shape)
print("word2vec dataset:", word2vec.shape)
print("bert dataset:    ", bert.shape)

# Audio Features
print("\n\nAudio Features\n")
## Mel Frequency Cepstral Coefficients (MFFCs)
print("mfcc bow dataset:  ", mfcc_bow.shape)
print("mfcc stats dataset:", mfcc_stats.shape)

## Essentia: An Audio Analysis Library for Music Information Retrieval
print("essentia dataset:  ", essentia.shape)

## Block Level features (BLFs)

print("blf correlation dataset:     ", blf_correlation.shape)
print("blf delta spectral dataset:  ", blf_deltaspectral.shape)
print("blf logfluc dataset:         ", blf_logfluc.shape)
print("blf spectral dataset:        ", blf_spectral.shape)
print("blf spectralcontrast dataset:", blf_spectralcontrast.shape)
print("blf vardeltaspectral dataset:", blf_vardeltaspectral.shape)


# Video Features
print("\n\nVideo Features\n")
## Derivative Content DC, Video Features
print("incp dataset:      ", incp.shape)
print("resnet dataset:    ", resnet.shape)
print("vgg19 dataset:     ", vgg19.shape)

Lyrics Features

tfidf dataset:    (68641, 1000)
word2vec dataset: (68641, 300)
bert dataset:     (68641, 768)


Audio Features

mfcc bow dataset:   (68641, 500)
mfcc stats dataset: (68641, 104)
essentia dataset:   (68641, 1034)
blf correlation dataset:      (68641, 1326)
blf delta spectral dataset:   (68641, 1372)
blf logfluc dataset:          (68641, 3626)
blf spectral dataset:         (68641, 980)
blf spectralcontrast dataset: (68641, 800)
blf vardeltaspectral dataset: (68641, 1344)


Video Features

incp dataset:       (68641, 4096)
resnet dataset:     (68641, 4096)
vgg19 dataset:      (68641, 8192)


In [11]:
def compute_in_batches_top(arr_a: np.array, arr_b: np.array, simfunction, idx_values, batches:int = 1, top: int=100):
    
    splits_b = np.array_split(arr_b, batches, axis=0)
    splits_idx = np.array_split(np.arange(arr_b.shape[0]), batches, axis=0)
    
    top_values = []
    init_state = 0
    for b,i in tqdm(list(zip(splits_b, splits_idx))):
        size_batch = b.shape[0]
        result = simfunction(arr_a, b).T
        result[(np.arange(size_batch),i)] = -1
        top_values.append(idx_values[np.argsort(result * -1, axis=1)][:,:top])
        init_state += b.shape[0]
              
    return np.concatenate(top_values, axis=0)

def generate_ids(df, simFunction, file_name):
    print(df.shape)
    top = compute_in_batches_top(
        df.to_numpy(dtype=np.float32), 
        df.to_numpy(dtype=np.float32), 
        simFunction, 
        df.index.values, 
        batches=100,   
        top=100)
     
    dt.Frame(pd.DataFrame(top , index=df.index.values).reset_index()).to_csv(file_name)
    del top

## Computation of Top 100 ids with different feature vectors cosine similarity

In [75]:
features = datasets.keys() 
for feature in features:
    file_name = f'./TopIds/top_ids_cosine_{feature}_complete.csv'
    print(file_name)
#Uncomment to generate ids
#     generate_ids(
#         datasets[feature], 
#         get_cosine_similarity,
#         file_name)

./TopIds/top_ids_cosine_tfidf_complete.csv
./TopIds/top_ids_cosine_word2vec_complete.csv
./TopIds/top_ids_cosine_bert_complete.csv
./TopIds/top_ids_cosine_mfcc_bow_complete.csv
./TopIds/top_ids_cosine_mfcc_stats_complete.csv
./TopIds/top_ids_cosine_essentia_complete.csv
./TopIds/top_ids_cosine_blf_delta_spectral_complete.csv
./TopIds/top_ids_cosine_blf_correlation_complete.csv
./TopIds/top_ids_cosine_blf_logfluc_complete.csv
./TopIds/top_ids_cosine_blf_spectral_complete.csv
./TopIds/top_ids_cosine_blf_spectral_contrast_complete.csv
./TopIds/top_ids_cosine_blf_vardelta_spectral_complete.csv
./TopIds/top_ids_cosine_incp_complete.csv
./TopIds/top_ids_cosine_vgg19_complete.csv
./TopIds/top_ids_cosine_resnet_complete.csv


## TOP 100 Ids using each song as a query with jaccard similarity

In [76]:
features = datasets.keys() 
for feature in features:
    file_name = f'./TopIds/top_ids_jaccard_{feature}_complete.csv'
    print(file_name)
#Uncomment to generate ids
#     generate_ids(
#         datasets[feature], 
#         get_jaccard_similarity,
#         file_name)

./TopIds/top_ids_jaccard_tfidf_complete.csv
./TopIds/top_ids_jaccard_word2vec_complete.csv
./TopIds/top_ids_jaccard_bert_complete.csv
./TopIds/top_ids_jaccard_mfcc_bow_complete.csv
./TopIds/top_ids_jaccard_mfcc_stats_complete.csv
./TopIds/top_ids_jaccard_essentia_complete.csv
./TopIds/top_ids_jaccard_blf_delta_spectral_complete.csv
./TopIds/top_ids_jaccard_blf_correlation_complete.csv
./TopIds/top_ids_jaccard_blf_logfluc_complete.csv
./TopIds/top_ids_jaccard_blf_spectral_complete.csv
./TopIds/top_ids_jaccard_blf_spectral_contrast_complete.csv
./TopIds/top_ids_jaccard_blf_vardelta_spectral_complete.csv
./TopIds/top_ids_jaccard_incp_complete.csv
./TopIds/top_ids_jaccard_vgg19_complete.csv
./TopIds/top_ids_jaccard_resnet_complete.csv


## Generate Baseline

The baseline that we took in consideration is a random selection of songs

In [43]:
top_random_ids = np.empty((68641, 100))
print(top_random_ids.shape)
np.random.seed(42)
for i in tqdm(range(68641)):
    top_random_ids[i] = np.random.choice(68641, 100,replace=False)

(68641, 100)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 68641/68641 [00:55<00:00, 1243.23it/s]


In [62]:
top_random_ids[0,:5]

array([51329., 19980., 15110., 63047.,  9736.])

In [63]:
baseline_topids = np.take(tf_idf.index.values, top_random_ids.astype(int), axis=0)
baseline_topids[0,:5]

array(['zCRCrMUDW7Zg4NrA', 'vooe9Ftwnnx7UhQw', 'iChfD2M03EUdi0xE',
       'Cl7k1xDo7R5M02vn', 'z21tWq5rTMGe6h02'], dtype=object)

In [64]:
baseline_topids.shape

(68641, 100)

In [66]:
dt.Frame(pd.DataFrame(baseline_topids, index=tf_idf.index.values ).reset_index()).to_csv('./TopIdsTask2/top_ids_baseline_complete.csv')

## Other early fusion datasets

### Computations concatenating each dataset and then compute the metrics

In [30]:
audio_jaccard = ["essentia", "blf_logfluc", "mfcc_stats"]
lyrics_jaccard = ["bert", "tfidf"]
video_jaccard = ["vgg19", "resnet"]

In [77]:
for lyrics in lyrics_jaccard:
    for audio in audio_jaccard:
        for video in video_jaccard:
            file_name = f'./TopIdsFusion/top_ids_jaccard_earlyfusion_{lyrics}_{audio}_{video}_complete.csv'
            print(file_name)
#             generate_ids(datasets[lyrics].join(datasets[audio], on='id').join(datasets[video], on="id"),
#                          get_jaccard_similarity,
#                          file_name)

./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_essentia_vgg19_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_essentia_resnet_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_blf_logfluc_vgg19_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_blf_logfluc_resnet_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_mfcc_stats_vgg19_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_mfcc_stats_resnet_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_essentia_vgg19_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_essentia_resnet_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_blf_logfluc_vgg19_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_blf_logfluc_resnet_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_mfcc_stats_vgg19_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_mfcc_stats_resnet_complete.csv


In [15]:
audio_cosine = ["blf_spectral", "blf_logfluc", "mfcc_bow"]
lyrics_cosine = ["bert", "tfidf"]
video_cosine = ["incp", "resnet"]

In [78]:
for lyrics in lyrics_cosine:
    for audio in audio_cosine:
        for video in video_cosine:
            file_name = f'./TopIdsFusion/top_ids_cosine_earlyfusion_{lyrics}_{audio}_{video}_complete.csv'
            print(file_name)
#             generate_ids(datasets[lyrics].join(datasets[audio], on='id').join(datasets[video], on="id"),
#                          get_cosine_similarity,
#                          file_name)

./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_spectral_incp_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_spectral_resnet_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_logfluc_incp_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_logfluc_resnet_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_mfcc_bow_incp_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_mfcc_bow_resnet_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_spectral_incp_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_spectral_resnet_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_logfluc_incp_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_logfluc_resnet_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_mfcc_bow_incp_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_mfcc_bow_resnet_complete.csv


### Computations normalizing first each dataset and concatenating them

In [64]:
def normdatasets_and_join(d1, d2, d3):
    d1_norm = pd.DataFrame(normalize(d1.to_numpy(), norm='l2', axis=1), index=d1.index.values)
    d2_norm = pd.DataFrame(normalize(d2.to_numpy(), norm='l2', axis=1), index=d2.index.values)
    d3_norm = pd.DataFrame(normalize(d3.to_numpy(), norm='l2', axis=1), index=d3.index.values)
    return d1.join(d2, on='id').join(d3, on="id")

In [69]:
audio_cosine = ["blf_spectral", "blf_logfluc", "mfcc_bow"]
lyrics_cosine = ["bert", "tfidf"]
video_cosine = ["incp", "resnet"]

In [79]:
for lyrics in lyrics_cosine:
    for audio in audio_cosine:
        for video in video_cosine:
            file_name = f'./TopIdsFusion/top_ids_cosine_earlyfusion_{lyrics}_{audio}_{video}_normalized_complete.csv'
            print(file_name)
#             generate_ids(normdatasets_and_join(datasets[lyrics], datasets[audio], datasets[video]),
#                          get_cosine_similarity,
#                          file_name)

./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_spectral_incp_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_spectral_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_logfluc_incp_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_blf_logfluc_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_mfcc_bow_incp_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_bert_mfcc_bow_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_spectral_incp_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_spectral_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_logfluc_incp_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_blf_logfluc_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_cosine_earlyfusion_tfidf_mfcc_bow_incp_normalized_complete.csv
./TopIdsFusion/top_ids

In [80]:
audio_jaccard = ["essentia", "blf_logfluc", "mfcc_stats"]
lyrics_jaccard = ["bert", "tfidf"]
video_jaccard = ["vgg19", "resnet"]

In [81]:
for lyrics in lyrics_jaccard:
    for audio in audio_jaccard:
        for video in video_jaccard:
            file_name = f'./TopIdsFusion/top_ids_jaccard_earlyfusion_{lyrics}_{audio}_{video}_normalized_complete.csv'
            print(file_name)
#             generate_ids(normdatasets_and_join(datasets[lyrics], datasets[audio], datasets[video]),
#                          get_jaccard_similarity,
#                          file_name)

./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_essentia_vgg19_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_essentia_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_blf_logfluc_vgg19_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_blf_logfluc_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_mfcc_stats_vgg19_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_bert_mfcc_stats_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_essentia_vgg19_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_essentia_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_blf_logfluc_vgg19_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_blf_logfluc_resnet_normalized_complete.csv
./TopIdsFusion/top_ids_jaccard_earlyfusion_tfidf_mfcc_stats_vgg19_normalized_complete.csv
./TopIdsFusion/

### Late fusion only combine 3 features

In [14]:
def compute_top_late(d1: np.array, d2: np.array, d3: np.array, simfunction, idx_values, batches:int = 1, top: int=100):
    
    sx, sy = d1.shape
    splits_1 = np.array_split(d1, batches, axis=0)
    splits_2 = np.array_split(d2, batches, axis=0)
    splits_3 = np.array_split(d3, batches, axis=0)
    splits_idx = np.array_split(np.arange(sx), batches, axis=0)

    top_values = []
    for b1,b2,b3,i in tqdm(list(zip(splits_1, splits_2, splits_3, splits_idx))):
        size_batch = b1.shape[0]
        # Calculate similarities
        result1 = simfunction(d1, b1).T
        result2 = simfunction(d2, b2).T
        result3 = simfunction(d3, b3).T
        # Normalize similarities      
        n1 = np.linalg.norm(result1 , axis=1, keepdims=True)
        n2 = np.linalg.norm(result2 , axis=1, keepdims=True)
        n3 = np.linalg.norm(result3 , axis=1, keepdims=True)
        result1 =  np.divide(result1, n1, out=np.zeros_like(result1), where=n1!=0)
        result2 =  np.divide(result2, n2, out=np.zeros_like(result2), where=n2!=0)
        result3 =  np.divide(result3, n3, out=np.zeros_like(result3), where=n3!=0)
        
        # Combine similarities
        result_combined = np.sum([result1, result2, result3], axis=0)
        # Get Tops
        result_combined[(np.arange(size_batch),i)] = -1
        top_values.append(idx_values[np.argsort(result_combined * -1, axis=1)][:,:top])

    return np.concatenate(top_values, axis=0)

In [15]:
def generate_ids_late_fusion(df1, df2, df3, simFunction, file_name, b):
    indexes = df1.index.values
    top = compute_top_late(
        df1.loc[indexes].to_numpy(dtype=np.float32), 
        df2.loc[indexes].to_numpy(dtype=np.float32), 
        df3.loc[indexes].to_numpy(dtype=np.float32), 
        simFunction, 
        indexes, 
        batches=b,   
        top=100)
     
    dt.Frame(pd.DataFrame(top , index=indexes).reset_index()).to_csv(file_name)
    del top, indexes

In [16]:
audio_jaccard = ["essentia", "blf_logfluc", "mfcc_stats"]
# audio_jaccard = ["blf_logfluc", "mfcc_stats"]
lyrics_jaccard = ["bert", "tfidf"]
video_jaccard = ["vgg19", "resnet"]

In [17]:
for lyrics in lyrics_jaccard:
    for audio in audio_jaccard:
        for video in video_jaccard:
            file_name = f'./TopIdsFusion/top_ids_jaccard_latefusion_{lyrics}_{audio}_{video}_complete.csv'
            print(file_name)
            generate_ids_late_fusion(
                datasets[lyrics],
                datasets[audio],
                datasets[video],
                get_jaccard_similarity,
                file_name,
                200
            )

./TopIdsFusion/top_ids_jaccard_latefusion_bert_essentia_vgg19_complete.csv


100%|█████████████████████████████████████████| 200/200 [18:33<00:00,  5.57s/it]


./TopIdsFusion/top_ids_jaccard_latefusion_bert_essentia_resnet_complete.csv


100%|█████████████████████████████████████████| 200/200 [13:52<00:00,  4.16s/it]


./TopIdsFusion/top_ids_jaccard_latefusion_bert_blf_logfluc_vgg19_complete.csv


...

 57%|███████████████████████▎                 | 114/200 [09:35<07:32,  5.26s/it]/Users/eduardohdz/Documents/JKU/Semester1/MMSR_REPO/project/functions.py:28: RuntimeWarning: invalid value encountered in divide
  r = dotp / (divisor-dotp)
100%|█████████████████████████████████████████| 200/200 [18:04<00:00,  5.42s/it]


./TopIdsFusion/top_ids_jaccard_latefusion_tfidf_mfcc_stats_vgg19_complete.csv


100%|█████████████████████████████████████████| 200/200 [17:56<00:00,  5.38s/it]


./TopIdsFusion/top_ids_jaccard_latefusion_tfidf_mfcc_stats_resnet_complete.csv


100%|█████████████████████████████████████████| 200/200 [13:24<00:00,  4.02s/it]


In [18]:
audio_cosine = [ "blf_logfluc", "blf_spectral","mfcc_bow"]
lyrics_cosine = ["bert", "tfidf"]
video_cosine = ["incp", "resnet"]

In [19]:
for lyrics in lyrics_cosine:
    for audio in audio_cosine:
        for video in video_cosine:
            file_name = f'./TopIdsFusion/top_ids_cosine_latefusion_{lyrics}_{audio}_{video}_complete.csv'
            print(file_name)
            generate_ids_late_fusion(
                datasets[lyrics],
                datasets[audio],
                datasets[video],
                get_cosine_similarity,
                file_name,
                130
            )

./TopIdsFusion/top_ids_cosine_latefusion_bert_blf_logfluc_incp_complete.csv


100%|█████████████████████████████████████████| 130/130 [16:55<00:00,  7.81s/it]


./TopIdsFusion/top_ids_cosine_latefusion_bert_blf_logfluc_resnet_complete.csv


100%|█████████████████████████████████████████| 130/130 [16:27<00:00,  7.59s/it]


./TopIdsFusion/top_ids_cosine_latefusion_bert_blf_spectral_incp_complete.csv


...

100%|█████████████████████████████████████████| 130/130 [12:39<00:00,  5.84s/it]


./TopIdsFusion/top_ids_cosine_latefusion_tfidf_mfcc_bow_incp_complete.csv


100%|█████████████████████████████████████████| 130/130 [13:20<00:00,  6.16s/it]


./TopIdsFusion/top_ids_cosine_latefusion_tfidf_mfcc_bow_resnet_complete.csv


100%|█████████████████████████████████████████| 130/130 [12:45<00:00,  5.89s/it]
